In [1]:
import argparse, os, copy, random, sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.cluster import AffinityPropagation
from sklearn.mixture import GaussianMixture
from functools import partial

from tqdm import *

import dataset, utils_CGCD, losses, net
from net.resnet import *

from models.modelgen import ModelGen, ModelGen_new
from torchsummary import summary
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import tensorflow as tf
from covmet import ConvNet
from edison_functions import compute_euclidean, contrastive_loss, extract_sample
torch.manual_seed(1)
np.set_printoptions(threshold=sys.maxsize)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# device = 'cpu'

In [2]:
class MyDict(dict):
    def __init__(self, args):
        self.args

class BlankClass:
    pass

args = BlankClass()

args.sz_batch  = 512
args.nb_epochs = 60
args.nb_workers = 8

args.lr = 1e-4
args.weight_decay = 1e-4
args.lr_decay_step = 5
args.lr_decay_gamma =  0.5
args.warm = 5
args.bn_freeze = False 
args.dataset = 'pamap'   # Dataset 
only_test_step1 = False            # Just to test the data on Train_1
args.model = 'resnet18'        # Model Name
window_len = 200

args.sz_embedding = 128

In [3]:
pth_rst = 'CGCD-main/src/result/' + args.dataset
os.makedirs(pth_rst, exist_ok=True)
pth_rst_exp = 'Saved_Models/Initial/Edison/' + args.dataset + '/' #+ args.model + '_sp_' + str(args.use_split_modlue) + '_gm_' + str(args.use_GM_clustering) + '_' + args.exp
os.makedirs(pth_rst_exp, exist_ok=True)

print("Dataset :", args.dataset)
####

pth_dataset = '../datasets'

if args.dataset =='wisdm':
    pth_dataset = 'HAR_data/Wisdm/'
elif args.dataset =='realworld':
    pth_dataset = 'HAR_data/realworld/'
    nb_classes_now = 8
elif args.dataset =='oppo':
    pth_dataset = 'HAR_data/oppo/'
elif args.dataset =='pamap':
    pth_dataset = 'HAR_data/pamap/'

Dataset : pamap


In [4]:
dset_tr_0 = dataset.load(name=args.dataset, root=pth_dataset, mode='train_0', windowlen= window_len, autoencoderType= None)
dlod_tr_0 = torch.utils.data.DataLoader(dset_tr_0, batch_size=args.sz_batch, shuffle=True, num_workers=args.nb_workers, drop_last=True)

dset_ev = dataset.load(name=args.dataset, root=pth_dataset, mode='eval_0', windowlen=window_len, autoencoderType= None)
dlod_ev = torch.utils.data.DataLoader(dset_ev, batch_size=args.sz_batch, shuffle=False, num_workers=args.nb_workers)

dset_test_0 = dataset.load(name=args.dataset, root=pth_dataset, mode='test_0', windowlen=window_len, autoencoderType= None)
dlod_test_0 = torch.utils.data.DataLoader(dset_test_0, batch_size=args.sz_batch, shuffle=False, num_workers=args.nb_workers)

# nb_classes = dset_test_0.nb_classes()
nb_classes = dset_tr_0.nb_classes()

# Configuration for the Model
if(args.model == 'resnet18'):
    cfg = {'weights_path': 'CGCD-main/src/Saved_Models/UK_BioBank_pretrained/mtl_best.mdl', "use_ssl_weights" : False, 'conv_freeze': False, 'load_finetuned_mtl': False,
        'checkpoint_name' :'', 'epoch_len': 2, 'output_size': '', 'embedding_dim': args.sz_embedding, 'bottleneck_dim': None,
            'output_size':nb_classes,'weight_norm_dim': ''}

    model = ModelGen_new(cfg).create_model().to(device)

if(args.model == 'harnet'):
    repo = 'OxWearables/ssl-wearables'
    model = torch.hub.load(repo, 'harnet5', class_num=nb_classes, pretrained=True).to(device)
    del model.classifier
    model.embedding = nn.Sequential(nn.Linear(model.feature_extractor.layer5[0].out_channels,args.sz_embedding)).to(device)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11} {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
With Embedding
No of classes : -> 12


In [5]:
dset_tr_0.xs[0].shape

(200, 9)

In [6]:
gpu_id =0
param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.embedding.parameters()))) if gpu_id != -1 else list(set(model.module.parameters()).difference(set(model.module.model.embedding.parameters())))},
    {'params': model.embedding.parameters() if gpu_id != -1 else model.embedding.parameters(), 'lr': float(args.lr) * 1},]

    # param_groups.append({'params': criterion_pa.parameters(), 'lr': float(args.lr)*100 })
opt_pa = torch.optim.AdamW(param_groups, lr=float(args.lr), weight_decay=args.weight_decay)


In [7]:
version = 1
step =0
pth_pth = '{}{}_{}_best_windowlen_{}_embedding_size_{}_version_{}_step_{}.pth'.format(pth_rst_exp, args.dataset, args.model, window_len, args.sz_embedding, version, step)
checkpoint = torch.load(pth_pth)

model.load_state_dict(checkpoint['model_pa_state_dict'])

<All keys matched successfully>

In [8]:
loaded_prototypes = np.load('{}{}_{}_prototypes_windowlen_{}_embedding_size_{}_version_{}_step_{}.npy'.format(pth_rst_exp, args.dataset, args.model, window_len, args.sz_embedding, version, step), allow_pickle= True)

trained_prototypes_dict = dict()
for loaded_prototype in loaded_prototypes:
    trained_prototypes_dict[int(loaded_prototype[0])] = loaded_prototype[1] 

In [9]:
z_proto_trained = torch.from_numpy(np.array(list(trained_prototypes_dict.values()))).float().to(device)

# Calculating Test Accuracy
# with torch.no_grad():
#     total_accuracy_test, n_steps = 0, 0
#     for x, y, z in dlod_test_0:
#         embeddings_test = model(x.to(device))
#         dists_test = compute_euclidean(embeddings_test,z_proto_trained)
#         log_p_test = F.softmax(-dists_test,dim=1)
#         test_accuracy = accuracy_score(torch.argmax(log_p_test, dim = 1).to('cpu').detach().numpy(), y.to('cpu').detach().numpy())
#         total_accuracy_test += test_accuracy
#         n_steps +=1

#     print("Test Accuracy {:.2f} %".format(total_accuracy_test/n_steps))

# Calculating Train Accuracy
# with torch.no_grad():
#     total_accuracy_train, n_steps = 0, 0
#     all_embeddings = []
#     all_ys = []
#     for x, y, z in dlod_tr_0:
#         embeddings_train = model(x.to(device))
#         if(len(all_embeddings)==0):
#             all_embeddings = embeddings_train
#             all_ys = y
#         else:
#             all_embeddings = torch.concatenate((all_embeddings, embeddings_train), axis =0)
#             all_ys = torch.concatenate((all_ys, y), axis =0)


#         dists_train = compute_euclidean(embeddings_train,z_proto_trained)
#         log_p_train = F.softmax(-dists_train,dim=1)
#         train_accuracy = accuracy_score(torch.argmax(log_p_train, dim = 1).to('cpu').detach().numpy(), y.to('cpu').detach().numpy())
#         total_accuracy_train += train_accuracy
#         n_steps +=1

#     print("Train Accuracy {:.2f} %".format(total_accuracy_train/n_steps))

In [10]:
dset_tr_now_md = 'train_1' # 'train_2'
dset_ev_now_md = 'eval_1' # 'eval_2'

dset_tr_now = dataset.load(name=args.dataset, root=pth_dataset, mode=dset_tr_now_md,windowlen= window_len, autoencoderType= None)
dset_ev_now = dataset.load(name=args.dataset, root=pth_dataset, mode=dset_ev_now_md,windowlen= window_len, autoencoderType= None)
dset_test = dataset.load(name=args.dataset, root=pth_dataset, mode='test_1', windowlen= window_len, autoencoderType= None)

dlod_tr_now = torch.utils.data.DataLoader(dset_tr_now, batch_size=args.sz_batch, shuffle=True, num_workers=args.nb_workers)
dlod_ev_now = torch.utils.data.DataLoader(dset_ev_now, batch_size=args.sz_batch, shuffle=False, num_workers=args.nb_workers)

dlod_test = torch.utils.data.DataLoader(dset_test, batch_size=args.sz_batch, shuffle=False, num_workers=args.nb_workers)

nb_classes_now = dset_tr_now.nb_classes()

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17} {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}


In [12]:
# Prediction accuracy on Stage 2 test dataset on Stage 1 Model.

predicted_y = []
actual_y = []
with torch.no_grad():
    z_proto_test = torch.from_numpy(np.array(list(trained_prototypes_dict.values()))).float().to(device)
    for x_test, y_test, z_test in dlod_test_0:
        embeddings_val = model(x_test.to(device))
        dists_val = compute_euclidean(embeddings_val,z_proto_test)
        log_p_val = F.softmax(-dists_val,dim=1)
        if(len(predicted_y)==0):
            predicted_y = torch.argmax(log_p_val, dim = 1).to('cpu').detach().numpy()
            actual_y = y_test.to('cpu').detach().numpy()
        else:
            predicted_y = np.concatenate((predicted_y, torch.argmax(log_p_val, dim = 1).to('cpu').detach().numpy()))
            actual_y = np.concatenate((actual_y,y_test.to('cpu').detach().numpy() ))
        
    

    y_test = torch.tensor(actual_y).type(torch.LongTensor)
    predicted_y = torch.tensor(predicted_y).type(torch.LongTensor)

    seen_classes = torch.where(y_test < nb_classes, 1, 0)
    seen_classes_idx = torch.nonzero(seen_classes)
    unseen_classes = torch.where(y_test >= nb_classes, 1, 0)
    unseen_classes_idx = torch.nonzero(unseen_classes)

    if(seen_classes.sum().item()>0):
        acc_o = accuracy_score(y_test[seen_classes_idx], predicted_y[seen_classes_idx])
    else:
        acc_o =0
    if(unseen_classes.sum().item()>0):
        acc_n = accuracy_score(y_test[unseen_classes_idx], predicted_y[unseen_classes_idx])
    else:
        acc_n =0

    
    print("Stage 2 Test Dataset on Stage 1 trained model, Seen Accuracy {:.2f}  Unseen Accuracy  {:.2f}".
          format(acc_o, acc_n))

Stage 2 Test Dataset on Stage 1 trained model, Seen Accuracy 0.80  Unseen Accuracy  0.00


In [14]:
def generate_dataset(dataset, index, index_target=None, target=None):
    dataset_ = copy.deepcopy(dataset)

    if target is not None:
        for i, v in enumerate(index_target):
            dataset_.ys[v] = target[i]

    for i, v in enumerate(index):
        # print("Index ",index, "Dataset.I",dataset_.I)
        j = v - i    # We seperate i because as we pop a element outside the array moves towards left and its size decreases
        dataset_.I.pop(j)
        dataset_.ys.pop(j)
        # dataset_.im_paths.pop(j)
    return dataset_

In [15]:
# Removing all the old indices from the dataloader.
old_indices = []
for idx,y in enumerate(dset_tr_now.ys):
    if(y < nb_classes):
        old_indices.append(idx)


dset_tr_only_new = generate_dataset(dset_tr_now, old_indices)
dlod_tr_only_new = torch.utils.data.DataLoader(dset_tr_only_new, batch_size=args.sz_batch, shuffle=True, num_workers=args.nb_workers)
# np.unique(dset_tr_o.ys)

In [16]:
from replay_memory import *
replay_size = 6

replay_buffer = ReplayMemory(replay_size)
replay_buffer.update((np.array(dset_tr_0.xs), np.array(dset_tr_0.ys)))

New classes: [2, 9, 7, 3, 0, 5, 10, 1, 6, 4, 8, 11], Old Classes: dict_keys([])
 Updated memory size for each old class: 6 [Train size: {11: 6, 4: 6, 8: 6, 7: 6, 5: 6, 9: 6, 1: 6, 6: 6, 2: 6, 0: 6, 3: 6, 10: 6}


In [17]:

model_now = copy.deepcopy(model)
model_now = model_now.to(device)
opt_pa_now = torch.optim.AdamW(model_now.parameters(), lr=float(args.lr), weight_decay=args.weight_decay)


In [15]:
from edison_functions import compute_euclidean, contrastive_loss, extract_sample
from sklearn import utils

step1_prototypes_dict = copy.deepcopy(trained_prototypes_dict)
counters = dict()
best_weighted_acc = 0
args.gpu_id = 0
for epoch in range(0,1):  #args.nb_epochs
    
    model_now.train()
    ####
    bn_freeze = args.bn_freeze
    if bn_freeze:
        modules = model_now.modules() if args.gpu_id != -1 else model_now.module.model.modules()
        for m in modules:
            if isinstance(m, nn.BatchNorm2d):
                m.eval()
    if args.warm > 0:
        # Early Stopping to remain one during the warnmp
        if args.gpu_id != -1:
            unfreeze_model_param = list(model_now.embedding.parameters()) 
        else:
            unfreeze_model_param = list(model_now.module.model.embedding.parameters()) 

        if epoch == 0:
            for param in list(set(model_now.parameters()).difference(set(unfreeze_model_param))):
                param.requires_grad = False
        if epoch == args.warm:
            for param in list(set(model_now.parameters()).difference(set(unfreeze_model_param))):
                param.requires_grad = True

    
    total_contrastive_loss = 0
    total_ce_loss = 0
    nsteps = 0
    for x, y, z in dlod_tr_only_new:
        
        y_map_labels = tf.keras.utils.to_categorical(y, num_classes=nb_classes_now)

        excluded_classes = []
        replay_set, replay_map_labels = replay_buffer.exemplar_train(excluded_classes)

        replay_set = torch.from_numpy(np.array(replay_set)).float()
        replay_set = torch.Tensor(np.transpose(replay_set, (0,2,1)))
        
        replay_map_labels_temp = torch.Tensor(replay_map_labels)  # Storing the non-binarized form
        replay_map_labels = torch.Tensor(replay_map_labels)
        replay_map_labels = tf.keras.utils.to_categorical(replay_map_labels, num_classes=nb_classes_now)

        query_set_x = torch.cat((x,replay_set),0)  # Merging old and new class elements
        query_set_labels = torch.cat((torch.Tensor(y_map_labels),torch.Tensor(replay_map_labels)),0).to(device)

    
        x_support_embeddings = model_now(x.to(device))
        x_query_embeddings = model_now(query_set_x.to(device))

        # Update/Create Prototypes of New Classes
        classes = np.sort(np.unique(y))
        for c in classes:
            if c in step1_prototypes_dict.keys():
                p_mean_old = copy.deepcopy(np.array(step1_prototypes_dict[c]).astype(np.float64))
                # print(c, np.shape(p_mean_old), p_mean_old)
                new_count = len(np.array(x_support_embeddings.data.cpu().numpy()[y==c]))
                #print(np.shape(p_mean_old), np.shape(np.array(X)))
                
                p_mean = float((counters[c]/(1.*(counters[c]+new_count))))*p_mean_old + np.sum(np.array(x_support_embeddings.data.cpu().numpy())[y==c],axis=0)/(counters[c]+new_count)
                #print(p_mean, p_mean_old)
                #sys.exit()
                step1_prototypes_dict[c] = copy.deepcopy(p_mean.flatten().astype(np.float64))
                counters[c] += new_count
                #print('old: ',c, p_mean_old, self.prototypes[c])

            else:
                # print('new: ', self.prototypes.keys(), c)
                p_mean = np.mean(x_support_embeddings.data.cpu().numpy()[y==c],axis=0)

                #print(np.shape(X[y==c]))
                step1_prototypes_dict[c] = copy.deepcopy(p_mean.flatten())
                counters[c] = len(x_support_embeddings.data.cpu().numpy()[y==c])

        # Calculate Distances.
        dists = torch.ones((len(x_query_embeddings),nb_classes_now))*float('inf')
        dists = dists.float().to(device)
        #print("CURRENT CLASSES IN PROTOTYPE MEMORY: ", list(self.memory.prototypes.keys()))
        for c in step1_prototypes_dict.keys():

            z_proto = torch.from_numpy(step1_prototypes_dict[c][None,:]).float().to(device)# Adding None just increases the shape from (128,) -> (1,128)
            dist = compute_euclidean(x_query_embeddings,z_proto)
            #print(np.shape(dist))
            dists[:,c] = torch.squeeze(dist)

        # Calculate the Loss.
        log_p = F.softmax(-dists,dim=1)
        ce_loss = F.binary_cross_entropy(log_p, query_set_labels) 

        contrastive_losses  = contrastive_loss(x_query_embeddings, query_set_labels,step1_prototypes_dict, balance=True)

        loss = 10*ce_loss + contrastive_losses
        
        total_contrastive_loss += contrastive_losses.item()
        total_ce_loss += ce_loss.item() 
        nsteps +=1 
        
        
        opt_pa_now.zero_grad()
        loss.backward()
        opt_pa_now.step()
        
        # Online Training.
        online_epochs = 1
        total_online_training_ce_loss = 0
        total_online_training_contrastive_loss = 0
        for i in range(1,online_epochs):
            query_set1, query_map_labels1 = utils.shuffle(query_set_x,query_set_labels, random_state=i)
            x_query_embeddings_1 = model_now(query_set1.to(device))

            dists_1 = torch.ones((len(x_query_embeddings_1),nb_classes_now))*float('inf')
            
            dists_1 = dists_1.float().to(device)
            #print("CURRENT CLASSES IN PROTOTYPE MEMORY: ", list(self.memory.prototypes.keys()))
            for c in step1_prototypes_dict.keys():

                z_proto_1 = torch.from_numpy(step1_prototypes_dict[c][None,:]).float().to(device)# Adding None just increases the shape from (128,) -> (1,128)
                dist_1 = compute_euclidean(x_query_embeddings_1,z_proto_1)
                #print(np.shape(dist))
                dists_1[:,c] = torch.squeeze(dist_1)
                
            log_p_1 = F.softmax(-dists_1,dim=1)

            ce_loss1 = F.binary_cross_entropy(log_p_1, query_map_labels1)
            
            contrastive_losses = contrastive_loss(x_query_embeddings_1, query_map_labels1, step1_prototypes_dict, balance=True)  
            loss = 10*ce_loss1 + contrastive_losses
            opt_pa_now.zero_grad()
            loss.backward()
            opt_pa_now.step()

        
        
        # Updating the prototypes of the seen/base classes
    model_now.eval()
    if len(replay_set) > 0:
        
        base_new_prototypes = dict()
        base_new_counters = dict()
        #print(classes)
        replay_set_embeddings = model_now(replay_set.to(device))
        old_classes = np.unique(replay_map_labels_temp.data.cpu())
        for c in old_classes:
            p_mean = replay_set_embeddings[replay_map_labels_temp.data.cpu()==c].mean(0)
            #print(np.shape(X[y==c]))
            base_new_prototypes[c] = copy.deepcopy(list(p_mean.data.cpu().numpy().flatten()))
            base_new_counters[c] = len(replay_set_embeddings[replay_map_labels_temp.data.cpu()==c])

        momentum = 0.5
        for c in base_new_prototypes.keys():
            step1_prototypes_dict[c] = copy.deepcopy((momentum*np.array(step1_prototypes_dict[c]).astype(np.float64) + (1.-momentum)*np.array(base_new_prototypes[c]).astype(np.float64)).flatten().astype(np.float64))


    print("Epoch {} CE Loss {:.5f} Constastive Loss {:.5f}".format(epoch, total_ce_loss/nsteps, total_contrastive_loss/nsteps ))


    print('==> Evaluation..')
    
    predicted_y = []
    actual_y = []
    with torch.no_grad():
        z_proto_val = torch.from_numpy(np.array(list(step1_prototypes_dict.values()))).float().to(device)
        for x_val, y_val, z_val in dlod_ev_now:
            embeddings_val = model_now(x_val.to(device))
            dists_val = compute_euclidean(embeddings_val,z_proto_val)
            log_p_val = F.softmax(-dists_val,dim=1)
            if(len(predicted_y)==0):
                predicted_y = torch.argmax(log_p_val, dim = 1).to('cpu').detach().numpy()
                actual_y = y_val.to('cpu').detach().numpy()
            else:
                predicted_y = np.concatenate((predicted_y, torch.argmax(log_p_val, dim = 1).to('cpu').detach().numpy()))
                actual_y = np.concatenate((actual_y,y_val.to('cpu').detach().numpy() ))
            
        

        y_val = torch.tensor(actual_y).type(torch.LongTensor)
        predicted_y = torch.tensor(predicted_y).type(torch.LongTensor)

        seen_classes = torch.where(y_val < nb_classes, 1, 0)
        seen_classes_idx = torch.nonzero(seen_classes)
        unseen_classes = torch.where(y_val >= nb_classes, 1, 0)
        unseen_classes_idx = torch.nonzero(unseen_classes)

        if(seen_classes.sum().item()>0):
            acc_o = accuracy_score(y_val[seen_classes_idx], predicted_y[seen_classes_idx])
        else:
            acc_o =0
        if(unseen_classes.sum().item()>0):
            acc_n = accuracy_score(y_val[unseen_classes_idx], predicted_y[unseen_classes_idx])
        else:
            acc_n =0
                    
        acc_a = accuracy_score(y_val, predicted_y)

        print("Valid Accuracies Seen {:.2f}, Unseen {:.2f}, Overall {:.2f}".format(acc_o, acc_n, acc_a))


        step = 2
        if((acc_o + acc_n + acc_a)/3 > best_weighted_acc):
            best_seen = acc_o
            best_unseen = acc_n
            best_overall = acc_a
            best_weighted_acc = (best_seen + best_unseen + best_overall)/3
            es_count = 0
            print("Got Better Model with  Seen {:.2f} ,  Unseen Accuracies {:.2f},  Overalll {:.2f} and  Average as {:.2f} ".format( best_seen,best_unseen, best_overall,best_weighted_acc))
            # torch.save({'model_pa_state_dict': model_now.state_dict(), 'proxies_param': criterion_pa_now.proxies}, '{}{}_{}_model_last_windowlen_{}_sz_embedding_{}_alpha{}_mrg_{}_step_{}_epoch_{}.pth'.format(pth_rst_exp, args.dataset, args.model,window_len, args.sz_embedding, args.alpha, args.mrg, str(step), epoch))
        else:                                                                                                     
            es_count +=1 
            print("Early Stopping Count ", es_count)
            
        if(epoch< args.warm):
            es_count =0    # If we are having warmup then no early stopping
        
        if(es_count ==10):
            
            print("Best  Valid Accuracies Seen {:.2f} , and Unseen {:.2f},  Overall {:.2f}, Average {:.2f} ".format( best_seen, best_unseen, best_overall, best_weighted_acc))
            print("Early Stopping")
            break

c:\Users\Dhruv\anaconda3\envs\CLHAR\Lib\site-packages\torch\nn\modules\conv.py:303: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),


Epoch 0 CE Loss 0.28710 Constastive Loss 5.78314
==> Evaluation..
Valid Accuracies Seen 0.50, Unseen 0.38, Overall 0.40
Got Better Model with  Seen 0.50 ,  Unseen Accuracies 0.38,  Overalll 0.40 and  Average as 0.43 


KeyboardInterrupt: 

In [ ]:
torch.unique(y_val)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [ ]:
# all_ys = all_ys.to('cpu').detach().numpy()

In [ ]:
# import matplotlib.pylab as plt
# plt.figure(figsize=(10,10))
# cm = plt.get_cmap('gist_rainbow')
# NUM_COLORS = len(trained_prototypes_dict.keys())

# colors = [cm((1.*i)/NUM_COLORS) for i in np.arange(NUM_COLORS)]
# markers=['.',  'x', 'h','1']

# keys = trained_prototypes_dict.keys()
# prototypes_pca[0]


# for k in keys:
#     plt.scatter(prototypes_pca[k][0], prototypes_pca[k][1], marker = 'o')
#     plt.scatter(emb_pca[np.array(all_ys) == k][:,0] , emb_pca[np.array(all_ys) == k][:,1], marker = 'o')

#     plt.annotate(k, (prototypes_pca[k,0], prototypes_pca[k,1]),
#                  horizontalalignment='center',
#                  verticalalignment='center',
#                  size=10, weight='bold',rotation=45,
#                  color='k')